***
# 04. 카운트 기반의 단어 표현(Count based word Representation)
***

자연어 처리에서 텍스트를 표현하는 방법으로는 여러가지 방법이 있습니다. 이번 챕터에서는 그 중 정보 검색과 텍스트 마이닝 분야에서 주로 사용되는 카운트 기반의 텍스트 표현 방법인 DTM(Document Term Matrix)과 TF-IDF(Term Frequency-Inverse Document Frequency)에 대해서 다룹니다.

텍스트를 위와 같은 방식으로 수치화를 하고나면, 통계적인 접근 방법을 통해 여러 문서로 이루어진 텍스트 데이터가 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내거나, 문서의 핵심어 추출, 검색 엔진에서 검색 결과의 순위 결정, 문서들 간의 유사도를 구하는 등의 용도로 사용할 수 있습니다.



## 04-01 다양한 단어의 표현 방법
***

### 1. 단어의 표현 방법
***

- **국소 표현(Local Representation) 방법**
    + 국소 표현 방법은 해당 단어 그 자체만 보고, 특정값을 맵핑하여 단어를 표현하는 방법
    + 이산 표현(Discrete Representation)이라고도 한다.
- **분산 표현(Distributed Representation) 방법**
    + 그 단어를 표현하고자 주변을 참고하여 단어를 표현하는 방법
    + 연속 표현(Continuous Represnetation)이라고 표현하기도 한다.

**이 두 방법의 차이** 
- 국소 표현 방법은 단어의 의미, 뉘앙스를 표현할 수 없다.
- 분산 표현 방법은 단어의 뉘앙스를 표현할 수 있게 됩니다.


### 2. 단어 표현의 카테고리화
***
아래와 같은 기준으로 단어 표현을 카테고리화하여 작성되었습니다.
![단어표현 카테고리화](img.png) 

이번 챕터의 **Bag of Words**는 **국소 표현**에(Local Representation)에 속하며, 단어의 빈도수를 카운트(Count)하여 단어를 수치화하는 단어 표현 방법입니다.
이 챕터에서는 **BoW**와 그의 확장인 **DTM**(또는 TDM)에 대해서 학습하고, 이러한 빈도수 기반 단어 표현에 단어의 중요도에 따른 가중치를 줄 수 있는 **TF-IDF**에 대해서 학습합니다.

## 04-02 Bag of Words(BoW)
***

### 1. Bag of Words란?
***
Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법입니다.
단어의 순서는 중요하지 않습니다.

BoW를 만드는 두 가지 과정
**(1)** 각 단어에 고유한 정수 인덱스를 부여합니다.  # 단어 집합 생성.
**(2)** 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.  


In [3]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
  # 온점 제거 및 형태소 분석
  document = document.replace('.', '')
  tokenized_document = okt.morphs(document)

  word_to_index = {}
  bow = []

  for word in tokenized_document:  
    if word not in word_to_index.keys(): # word_to_index에 단어가 없으면
      word_to_index[word] = len(word_to_index)  
      # BoW에 전부 기본값 1을 넣는다.
      bow.insert(len(word_to_index) - 1, 1) # 인덱스 0부터
      # 위에서 word_to_index에 값 넣어주었기 때문에 길이가 1 되어서 bow.insert 할 때는 인덱스 0부터 넣어주기 위해서 -1 해준다.
    else:
      # 재등장하는 단어의 인덱스
      index = word_to_index.get(word)
      # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
      bow[index] = bow[index] + 1

  return word_to_index, bow

In [4]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab) # word_to_index
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


### 2. Bag of Words의 다른 예제들
***

In [5]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [6]:
# 문서1과 문서2를 합쳐서 문서 3이라고 명명하고, BoW를 만들 수도 있다.

doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


BoW는 종종 여러 문서의 단어 집합을 합친 뒤에, 해당 단어 집합에 대한 각 문서의 BoW를 구하기도 한다. 가령, 문서3에 대한 단어 집합을 기준으로 문서1, 문서2의 BoW를 만든다고 한다면 결과는 아래와 같다.
***
#### 문서3 단어 집합에 대한 문서1 BoW : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  
#### 문서3 단어 집합에 대한 문서2 BoW : [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1]  
***
문서3 단어 집합에서 물가상승률이라는 단어는 인덱스가 4에 해당된다. 물가상승률이라는 단어는 문서1에서는 2회 등장하며, 문서2에서는 1회 등장하였기 때문에 두 BoW의 인덱스 4의 값은 각각 2와 1이 되는 것을 볼 수 있다.

BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이므로 주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단하는 작업에 쓰인다.
즉, 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰인다.
가령, '달리기', '체력', '근력'과 같은 단어가 자주 등장하면 해당 문서를 체육 관련 문서로 분류할 수 있다.

### 3. CountVectorizer 클래스로 BoW 만들기
***


In [3]:
# 사이킷 런에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원한다.

from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer() # 기본적으로 길이가 2이상인 것만 인식(정제작업)

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()[0]) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',sorted(vector.vocabulary_.items()))

bag of words vector : [1 1 2 1 2 1]
vocabulary : [('because', 0), ('know', 1), ('love', 2), ('want', 3), ('you', 4), ('your', 5)]


**주의할 점** 
CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다.
이는 한국어에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않음을 의미한다.

EX) 
**'정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'** 라는 문장을 CountVectorizer를 사용하여 BoW로 만들 경우
- CountVectorizer는 '물가상승률'이라는 단어를 인식하지 못 한다.
- CountVectorizer는 띄어쓰기를 기준으로 분리한 뒤에 '물가상승률과'와 '물가상승률은' 으로 조사를 포함해서 하나의 단어로 판단하기 때문에 서로 다른 두 단어로 인식한다.
- 그리고 '물가상승률과'와 '물가상승률은'이 각자 다른 인덱스에서 1이라는 빈도의 값을 갖게 된다.

### 4. 불용어를 제거한 BoW 만들기
***

In [8]:
# 영어의 BoW를 만들기 위해 사용하는 CountVectorizer는 불용어를 지정하면
# 불용어는 제외하고 BoW를 만들 수 있도록 불용어 제거 기능을 지원한다.

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

#### (1) 사용자가 직접 정의한 불용어 사용

In [9]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### (2) CountVectorizer에서 제공하는 자체 불용어 사용

In [10]:
# 영어의 BoW를 만들기 위해 사용하는 CountVectorizer는 불용어를 지정하면
# 불용어는 제외하고 BoW를 만들 수 있도록 불용어 제거 기능을 지원한다.
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english") # 'everything'을 불용어로 간주함
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


#### (3) NLTK에서 지원하는 불용어 사용

In [11]:
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


## 04-03 문서 단어 행렬(Document-Term Matrix, DTM)
***

**문서 단어 행렬(Document-Term Matrix, DTM)**
- 서로 다른 문서들의 BoW들을 결합한 표현 방법 (이하 DTM이라고 명명)
- 행과 열을 반대로 선택하면 TDM이라고 부르기도 합니다.
- 서로 다른 문서들을 비교할 수 있다.

### 1. 문서 단어 행렬(Document-Term Matrix, DTM)의 표기법
***

정의 : 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것

- 각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, 
- BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어

| 문서 | 내용           |
|---|--------------|
| 문서1 | 먹고 싶은 사과     |
| 문서2 | 먹고 싶은 바나나    |
| 문서3 | 길고 노란 바나나 바나나 |
| 문서4 | 저는 과일이 좋아요   |

**문서 단어 행렬로 표현**


![문서단어행렬](img_1.png)


각 문서에서 등장한 단어의 빈도를 행렬의 값으로 표기
문서 단어 행렬은 문서들을 서로 비교할 수 있도록 수치화할 수 있다는 점에서 의의를 갖는다.
만약 필요에 따라서는 형태소 분석기로 단어 토큰화를 수행하고, 불용어에 해당되는 조사들 또한 제거하여 더 정제된 DTM을 만들 수도 있다.

### 2. 문서 단어 행렬(Document-Term Matrix)의 한계
***

#### 1) 희소 표현(Sparse representation)

- 공간적 낭비와 계산 리소스를 증가시킬 수 있다.
- 만약 가지고 있는 전체 코퍼스가 방대한 데이터라면 문서 벡터의 차원은 수만 이상의 차원을 가질 수도 있다.
- 또한 많은 문서 벡터가 대부분의 값이 0을 가질 수도 있다.

#### 2) 단순 빈도 수 기반 접근

- 영어에 대해서 DTM을 만들었을 때, 불용어인 the는 어떤 문서이든 자주 등장할 수 밖에 없다.
- 유사한 문서인지 비교하고 싶은 문서1, 문서2, 문서3에서 동일하게 the가 빈도수가 높다고 해서 이 문서들이 유사한 문서라고 판단해서는 안 된다.


## 04-04 TF-IDF(Term Frequency-Inverse Document Frequency)
***

TF-IDF 가중치
- DTM 내에 있는 각 단어에 대한 중요도를 계산
- 기존의 DTM을 사용하는 것보다 보다 많은 정보를 고려하여 문서들을 비교할 수 있음

### 1. TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)
***

**TF-IDF(Term Frequency-Inverse Document Frequency)**
- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다.
- 우선 DTM을 만든 후, TF-IDF 가중치를 부여합니다.
- 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있습니다.


#### TF-IDF 식 이해하기

**TF-IDF는 TF와 IDF를 곱한 값을 의미**

문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때 TF, DF, IDF는 각각 다음과 같이 정의할 수 있다.

#### (1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.
DTM의 예제에서 각 단어들이 가진 값
각 문서에서의 각 단어의 등장 빈도를 나타내는 값

#### (2) df(t) : 특정 단어 t가 등장한 문서의 수.

오직 특정 단어 t가 등장한 문서의 수에만 관심을 가진다. 
앞서 배운 DTM에서 바나나는 문서2와 문서3에서 등장했기 때문에 이 경우, 바나나의 df는 2이다.


#### (3) idf(t) : df(t)에 반비례하는 수.


![idf(t) 수식](img_2.png)

 불용어 등과 같이 자주 쓰이는 단어들은 비교적 자주 쓰이지 않는 단어들보다 최소 수십 배 자주 등장한다.
 그런데 비교적 자주 쓰이지 않는 단어들조차 희귀 단어들과 비교하면 또 최소 수백 배는 더 자주 등장하는 편이다.
 이 때문에 log를 씌워주지 않으면, 희귀 단어들에 엄청난 가중치가 부여될 수 있다.
 로그를 씌우면 이런 격차를 줄이는 효과가 있다.
 

 **log 안의 식에서 분모에 1을 더해주는 이유 : 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위해서**


TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다.

- TF-IDF 값이 낮으면 중요도가 낮음
- TF-IDF 값이 크면 중요도가 큼 

즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 된다.

#### TF-IDF 이해하기


![문서단어행렬](img_1.png)


우선 TF는 앞서 사용한 DTM을 그대로 사용하면, 그것이 각 문서에서의 각 단어의 TF가 됩니다.
이제 구해야할 것은 TF와 곱해야할 값인 IDF입니다.
IDF 계산을 위해 사용하는 로그의 밑은 TF-IDF를 사용하는 사용자가 임의로 정할 수 있는데, 여기서 로그는 마치 기존의 값에 곱하여 값의 크기를 조절하는 상수의 역할을 합니다.
각종 프로그래밍 언어에서 패키지로 지원하는 TF-IDF의 로그는 대부분 자연 로그를 사용합니다. (자연 로그 -> ln)

![IDF표](img_3.png)


문서의 총 수는 4이기 때문에 ln 안에서 분자는 늘 4으로 동일합니다.
분모의 경우에는 각 단어가 등장한 문서의 수(DF)를 의미하는데, 예를 들어서 '먹고'의 경우에는 총 2개의 문서(문서1, 문서2)에 등장했기 때문에 2라는 값을 가집니다.
각 단어에 대해서 IDF의 값을 비교해보면 문서 1개에만 등장한 단어와 문서2개에만 등장한 단어는 값의 차이를 보입니다.
IDF는 여러 문서에서 등장한 단어의 가중치를 낮추는 역할을 하기 때문입니다.

**TF-IDF 계산결과**
각 단어의 TF는 DTM에서의 각 단어의 값과 같으므로, 앞서 사용한 DTM에서 단어 별로 위의 IDF값을 곱해주면 TF-IDF 값을 얻습니다.

![TF-IDF 계산결과](img_4.png)


문서3에서의 바나나만 TF 값이 2이므로 IDF에 2를 곱해주고, 나머진 TF 값이 1이므로 그대로 IDF 값을 가져오면 됩니다.
문서2에서의 바나나의 TF-IDF 가중치와 문서3에서의 바나나의 TF-IDF 가중치가 다른 것을 볼 수 있습니다.(TF가 각각 1과 2로 다름)
TF-IDF에서의 관점에서 보자면 TF-IDF는 특정 문서에서 자주 등장하는 단어는 그 문서 내에서 중요한 단어로 판단하기 때문입니다.
**문서2에서는 바나나를 한 번 언급했지만, 문서3에서는 바나나를 두 번 언급했기 때문에 문서3에서의 바나나를 더욱 중요한 단어라고 판단하는 것입니다.**

### 2. 파이썬으로 TF-IDF 직접 구현하기
***

In [2]:
# 앞의 설명에서 사용한 4개의 문서를 docs에 저장

import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print(vocab)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [3]:
# TF, IDF, 그리고 TF-IDF 값을 구하는 함수를 구현

# 총 문서의 수
N = len(docs) 

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

In [9]:
# TF를 구하기 : DTM을 데이터프레임에 저장하여 출력

result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([]) # 리스트 안에 리스트
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d)) # 위에서 어펜드 해준 []에다가 넣기

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [5]:
# 각 단어에 대한 IDF 값을 구하기

result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_ # 위에서 수기로 구한 IDF 값들과 정확히 일치

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [7]:
# TF-IDF 행렬 출력

result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


사실 실제 TF-IDF 구현을 제공하고 있는 많은 머신 러닝 패키지들은 패키지마다 식이 조금씩 상이하지만
위에서 배운 식과는 다른 조정된 식을 사용한다.

**위의 기본적인 식을 바탕으로 한 구현의 문제점**

- 전체 문서의 수 $n$이 4인데, $df(t)$의 값이 3인 경우 
$df(t)$에 1이 더해지면서 log항의 분자와 분모의 값이 같아지게 된다.
이는 $log$의 진수값이 1이 되면서 $idf(d,t)$의 값이 0이 됨을 의미.
(식 : $idf(d,t)=log(n/(df(t)+1))=0$)
IDF의 값이 0이라면 더 이상 가중치의 역할을 수행하지 못함.
  <br/>
- 사이킷런의 TF-IDF 구현체 또한 위의 식에서 조정된 식을 사용

### 3. 사이킷런을 이용한 DTM과 TF-IDF 실습
***

BoW를 설명하며 배운 CountVectorizer를 사용하여 DTM 만들기

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력 (인덱스 기준 정렬 시킴)
print(sorted(vector.vocabulary_.items()))


[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
[('do', 0), ('know', 1), ('like', 2), ('love', 3), ('should', 4), ('want', 5), ('what', 6), ('you', 7), ('your', 8)]


- 첫번째 열의 경우에는 0의 인덱스를 가진 do입니다.
do는 세번째 문서에만 등장했기 때문에, 세번째 행에서만 1의 값을 가집니다.
 <br/>

- 두번째 열의 경우에는 1의 인덱스를 가진 know입니다.
know는 첫번째 문서에만 등장했으므로 첫번째 행에서만 1의 값을 가집니다.


**사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공함**

사이킷런의 TF-IDF는 위에서 배웠던 보편적인 TF-IDF 기본 식에서 조정된 식을 사용합니다.
요약하자면, IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정하는 등의 차이로 TF-IDF가 가진 의도는 여전히 그대로 갖고 있습니다.

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)

print(sorted(tfidfv.vocabulary_.items())) # 인덱스 기준 정렬
print(sorted(tfidfv.vocabulary_))

[('do', 0), ('know', 1), ('like', 2), ('love', 3), ('should', 4), ('want', 5), ('what', 6), ('you', 7), ('your', 8)]
['do', 'know', 'like', 'love', 'should', 'want', 'what', 'you', 'your']


In [104]:
arr = tfidfv.transform(corpus).toarray()

df = pd.DataFrame(arr, columns = sorted(tfidfv.vocabulary_))
df

,do,know,like,love,should,want,what,you,your
0,0.00000,0.467351,0.000000,0.467351,0.00000,0.467351,0.00000,0.355432,0.467351
1,0.00000,0.000000,0.795961,0.000000,0.00000,0.000000,0.00000,0.605349,0.000000
2,0.57735,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.000000,0.000000
